## Problem 54 - Poker hands

https://projecteuler.net/problem=54

In [42]:
cardValue = {"T":10, "J":11, "Q":12, "K":13, "A":14}

def readHands(filename):
    with open(filename) as f:
        hands = []
        for hand in [ l.split(" ") for l in f.read().strip("\n").split("\n") ]:
            values = []
            for h in hand:
                v = h[0]
                if v in cardValue.keys():
                    v = cardValue[v]
                else:
                    v = int(v)
                values.append((v,h[1]))
            h1 = tuple(sorted(values[:5]))
            h2 = tuple(sorted(values[5:]))
            hands.append((h1,h2))
    return hands

In [43]:
handsTest = readHands("data/p054_poker_test.txt")
handsTest

[(((5, 'C'), (5, 'H'), (6, 'S'), (7, 'S'), (13, 'D')),
  ((2, 'C'), (3, 'S'), (8, 'D'), (8, 'S'), (10, 'D'))),
 (((5, 'D'), (8, 'C'), (9, 'S'), (11, 'S'), (14, 'C')),
  ((2, 'C'), (5, 'C'), (7, 'D'), (8, 'S'), (12, 'H'))),
 (((2, 'D'), (9, 'C'), (14, 'C'), (14, 'H'), (14, 'S')),
  ((3, 'D'), (6, 'D'), (7, 'D'), (10, 'D'), (12, 'D'))),
 (((4, 'D'), (6, 'S'), (9, 'H'), (12, 'C'), (12, 'H')),
  ((3, 'D'), (6, 'D'), (7, 'H'), (12, 'D'), (12, 'S'))),
 (((2, 'D'), (2, 'H'), (4, 'C'), (4, 'D'), (4, 'S')),
  ((3, 'C'), (3, 'D'), (3, 'S'), (9, 'D'), (9, 'S')))]

In [44]:
handsTest[0]

(((5, 'C'), (5, 'H'), (6, 'S'), (7, 'S'), (13, 'D')),
 ((2, 'C'), (3, 'S'), (8, 'D'), (8, 'S'), (10, 'D')))

In the card game poker, a hand consists of five cards and are ranked, from lowest to highest, in the following way:

- High Card: Highest value card.
- One Pair: Two cards of the same value.
- Two Pairs: Two different pairs.
- Three of a Kind: Three cards of the same value.
- Straight: All cards are consecutive values.
- Flush: All cards of the same suit.
- Full House: Three of a kind and a pair.
- Four of a Kind: Four cards of the same value.
- Straight Flush: All cards are consecutive values of same suit.
- Royal Flush: Ten, Jack, Queen, King, Ace, in same suit.

In [121]:
from collections import Counter

def handPoints(h):

    values = [ v for v,s in h ]
    suits  = [ s for v,s in h ]
    suitcount  = Counter(suits)
    valuecount = Counter(values)
    cv = Counter(valuecount.values())
    
    hasOnePair     = cv[2]==1 and cv[3]==0
    hasTwoPairs    = cv[2]==2
    hasThree       = cv[3]==1 and cv[2]==0
    hasFlush       = len(suitcount.keys())==1    
    hasStraight    = values[0]==values[1]-1==values[2]-2==values[3]-3==values[4]-4
    hasFullHouse   = cv[3]==1 and cv[2]==1
    hasFour        = cv[4]==1
    hasStraightFlush = hasStraight and hasFlush
    hasRoyalFlush    = hasStraight and hasFlush and highCard==14
    
    return ( hasRoyalFlush,    # 0
             hasStraightFlush, # 1
             hasFour,          # 2
             hasFullHouse,     # 3
             hasFlush,         # 4
             hasStraight,      # 5
             hasThree,         # 6
             hasTwoPairs,      # 7
             hasOnePair        # 8
           ) , valuecount

handPoints(handsTest[2][1])

((False, False, False, False, True, False, False, False, False),
 Counter({3: 1, 6: 1, 7: 1, 10: 1, 12: 1}))

In [122]:
def matchHands(h1,h2):
    
    values1 = [ v for v,s in h1 ]
    values2 = [ v for v,s in h2 ]
    
    points1,valuecount1 = handPoints(h1)
    points2,valuecount2 = handPoints(h2)
    
    for i,(p1,p2) in enumerate(zip(points1,points2)):

        #print(i,p1,p2)

        if p1 and not p2:
            return 1
        if p2 and not p1:
            return 2
        
        if p1 and p2: # Ties
            
            # There cannot be a tie with Royal Flush
            
            if i==1 or i==4 or i==5: # Straight Flush or Flush or Straight
                if max(values1) > max(values2):
                    return 1
                else:
                    return 2
    
            if i==2: # Four of a Kind
                v4_1 = [ v for v,c in valuecount1.items() if c==4 ][0]
                v4_2 = [ v for v,c in valuecount2.items() if c==4 ][0]
                if v4_1 > v4_2:
                    return 1
                else:
                    return 2
            
            if i==3 or i==6: # Full House or Three of a kind 
                v3_1 = [ v for v,c in valuecount1.items() if c==3 ][0]
                v3_2 = [ v for v,c in valuecount2.items() if c==3 ][0]
                if v3_1 > v3_2:
                    return 1
                else:
                    return 2
            
            if i==7: # Two pairs
                v2_1p = sorted([ v for v,c in valuecount1.items() if c==2 ],reverse=True)
                v2_2p = sorted([ v for v,c in valuecount2.items() if c==2 ],reverse=True)
                for v2_1,v2_2 in zip(v2_1p,v2_2p):
                    if v2_1 > v2_2:
                        return 1
                    elif v2_1 < v2_2:
                        return 2
                # if both pairs have same value, test 5th card value
                ve_1 = [ v for v,c in valuecount1.items() if c!=2 ][0]
                ve_2 = [ v for v,c in valuecount2.items() if c!=2 ][0]
                if ve_1 > ve_2:
                    return 1
                elif ve_1 < ve_2:
                    return 2
                
            if i==8: # Two of a kind
                v2_1 = [ v for v,c in valuecount1.items() if c==2 ][0]
                v2_2 = [ v for v,c in valuecount2.items() if c==2 ][0]
                if v2_1 > v2_2:
                    return 1
                elif v2_1 < v2_2:
                    return 2
                else: # pairs have same value, check other cards
                    for _ in range(2):
                        values1.pop(values1.index(v2_1))
                        values2.pop(values2.index(v2_2))
                    if max(values1) > max(values2):
                        return 1
                    else:
                        return 2
                    
    # no points, highest card wins
    if max(values1) > max(values2):
        return 1
    else:
        return 2
        
matchHands(handsTest[4][0],handsTest[4][1])

1

In [123]:
for h1,h2 in handsTest:
    print(matchHands(h1,h2))

2
1
2
1
1


In [126]:
hands = readHands("data/p054_poker.txt")

w1 = 0
#w2 = 0

for h1,h2 in hands:
    win = matchHands(h1,h2)
    if win==1:
        w1 += 1
    #if win==2:
    #   w2 += 1
   
#print(w1,w2,w1+w2)
print("Player 1 wins {} hands".format(w1))

Player 1 wins 376 hands
